# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
#pip install openai

In [2]:
import openai
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from typing import List, Union, Dict
from scipy.spatial.distance import cosine

In [3]:
open_api_key = 'YOUR_API_KEY'
openai.api_key = open_api_key

EMBEDDING_MODEL = "text-embedding-3-small"
BATCH_SIZE = 20
CSV_FILEPATH_WITH_EMBEDDINGS = 'data/election_phases_embeddings.csv'
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

In [4]:
def fetch_page(url: str):
    headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        return r.text
    else:
        print(r.status_code)
        return r.text

In [5]:
# Test if the function fetch the page correctly
test_url = "https://awbi.in/lok-sabha-election-schedule-2024/"
test_result = fetch_page(test_url)

In [6]:
# Extract phase numbers and text
phase_numbers = []
texts = []

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(test_result, "html.parser")

# Find all headers with class wp-block-heading containing "Phase"
phase_headers = soup.find_all("h3", class_="wp-block-heading")

# Initialize lists to store phase numbers and their corresponding text
phase_numbers = []
phase_texts = []

# Extract phase numbers and text
for header in phase_headers:
    phase_number = header.text.split()[1]  # Extract phase number from header text
    phase_numbers.append(phase_number)
    next_sibling = header.find_next_sibling("p")  # Find next sibling paragraph
    if next_sibling:
        phase_texts.append(next_sibling.get_text(strip=True))
    else:
        phase_texts.append("")  # Append empty string if no paragraph found

# Create a DataFrame using pandas
df = pd.DataFrame({
    'Phase Number': phase_numbers,
    'Text': phase_texts
})

# Display the DataFrame
(df)

,Phase Number,Text
0,1,The nominations of 1stphase will be done on 27...
1,2,Phase 2 of the elections will be conducted on ...
2,3,The 3rdphase of elections will held on 20 Apri...
3,4,The 4thphase of elections will take place on 1...
4,5,The nominations for this phase will be made on...
5,6,"The 6thphase of elections, which includes Unio..."
6,7,The 7thphase of elections will take place on 1...


In [7]:
# Setting display options for DataFrame
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_rows', None)

In [8]:
df

,Phase Number,Text
0,1,The nominations of 1stphase will be done on 27 March and conducted on 19 April. There are 21 states and union territories are included in this phase of elections.
1,2,Phase 2 of the elections will be conducted on 26 April because its nominations will end on 4 April. 12 states and union territories are included in this phase of Lok Sabha elections 2024
2,3,The 3rdphase of elections will held on 20 April. The nominations of this phase will be conducted on 7 May. 12 states and union territories are take part in nominations
3,4,The 4thphase of elections will take place on 13 May because its nominations end on 25 April. This phase includes the 10 states and union territories.
4,5,"The nominations for this phase will be made on 3 May. The 5thphase of the Lok Sabha elections 2024 will take place on 20 May, including the eight states and union territories."
5,6,"The 6thphase of elections, which includes Union territories and 7 states, will take place on 25 May. Nominations for this phase are ended on 6 May."
6,7,The 7thphase of elections will take place on 1 June because of the 8 states and the UT nomination ends on 14 May.


In [9]:
df.to_csv('data/election_phases.csv')

In [10]:
# where is csv file
CSV_FILEPATH = 'data/election_phases.csv'

# Read the DataFrame from CSV file
df = pd.read_csv(CSV_FILEPATH)

## Create Embeddings

In [11]:
#pip install tiktoken

In [12]:
# Function to get embeddings from OpenAI API
def get_embeddings(prompt: Union[str, List[str]], embedding_model: str) -> List[List[float]]:
    """
    Retrieves embeddings from OpenAI API for the given prompt using the specified embedding model.

    Args:
        prompt (Union[str, List[str]]): Input prompt or list of prompts.
        embedding_model (str): Name of the embedding model to use.

    Returns:
        List[List[float]]: List of embeddings for the input prompt(s).
    """
    response = openai.Embedding.create(
        input=prompt if isinstance(prompt, list) else [prompt],
        model=embedding_model
    )
    return [row.embedding for row in response.data]



In [13]:
# Function to create embeddings for DataFrame
def create_embeddings(df: pd.DataFrame, embedding_model_name: str = EMBEDDING_MODEL, batch_size: int = BATCH_SIZE) -> List[List[float]]:
    """
    Creates embeddings for the text data in the DataFrame using the specified embedding model.

    Args:
        df (pd.DataFrame): DataFrame containing text data.
        embedding_model_name (str): Name of the embedding model to use.
        batch_size (int): Size of batches for processing.

    Returns:
        List[List[float]]: List of embeddings corresponding to the text data.
    """
    embeddings_output = []
    for idx in range(0, len(df), batch_size):
        batch = df.iloc[idx:idx+batch_size]['Text'].tolist()
        embeddings = get_embeddings(batch, embedding_model_name)
        embeddings_output.extend(embeddings)
    return embeddings_output

In [14]:
# Add embeddings to DataFrame and save to CSV
df['embedding'] = create_embeddings(df)
df.to_csv(CSV_FILEPATH_WITH_EMBEDDINGS, sep=',', index=False)

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [41]:
df = pd.read_csv(CSV_FILEPATH_WITH_EMBEDDINGS)
df

Unnamed: 0  Phase Number  \
0           0             1   
1           1             2   
2           2             3   
3           3             4   
4           4             5   
5           5             6   
6           6             7   

                                                                                                                                                                                         Text  \
0                          The nominations of 1stphase will be done on 27 March and conducted on 19 April. There are 21 states and union territories are included in this phase of elections.   
1  Phase 2 of the elections will be conducted on 26 April because its nominations will end on 4 April. 12 states and union territories are included in this phase of Lok Sabha elections 2024   
2                     The 3rdphase of elections will held on 20 April. The nominations of this phase will be conducted on 7 May. 12 states and union territories are take part in nominations   
3                                       The 4thphase of elections will take place on 13 May because its nominations end on 25 April. This phase includes the 10 states and union territories.   
4             The nominations for this phase will be made on 3 May. The 5thphase of the Lok Sabha elections 2024 will take place on 20 May, including the eight states and union territories.   
5                                         The 6thphase of elections, which includes Union territories and 7 states, will take place on 25 May. Nominations for this phase are ended on 6 May.   
6                                                                           The 7thphase of elections will take place on 1 June because of the 8 states and the UT nomination ends on 14 May.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [42]:
import tiktoken
# Create a tokenizer that is designed to align with our embeddings
tokenizer = tiktoken.get_encoding("cl100k_base")

token_limit = 1000

In [43]:
def get_response(prompt: List[Dict[str, str]], client, model_name: str = COMPLETION_MODEL_NAME) -> str:
    response = openai.Completion.create(model=COMPLETION_MODEL_NAME,
                                        prompt=prompt,
                                        max_tokens=150)
    answer = response["choices"][0]["text"].strip()
    return answer

In [44]:
def basic_promt(USER_QUESTION: List[Dict[str, str]], client, model_name: str = COMPLETION_MODEL_NAME) -> str:
    # Use string formatting to complete the prompt
    prompt_template = """
    Answer the question based on the context below, and if the 
    question can't be answered based on the context, say 
    "I don't know"

    Context: 
    
    {}
    
    ---
    
    Question: {}
    Answer:"""
    context_list = []
    prompt = prompt_template.format(
        "\n\n###\n\n".join(context_list),
        USER_QUESTION
    )
    return prompt 

In [45]:
def custom_promt(USER_QUESTION: List[Dict[str, str]], client, model_name: str = COMPLETION_MODEL_NAME) -> str:
# Count the number of tokens in the prompt template and question
    prompt_template = """
    Answer the question based on the context below, and if the 
    question can't be answered based on the context, say 
    "I don't know"

    Context: 
    
    {}
    
    ---
    
    Question: {}
    Answer:"""
    token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(USER_QUESTION))
    
    # Create a list to store text for context
    context_list = []
    
    # Loop over rows of the sorted dataframe
    for text in df["Text"].values:
        
        # Append text to context_list if there is enough room
        token_count += len(tokenizer.encode(text))
        if token_count <= token_limit:
            context_list.append(text)
        else:
            # Break once we're over the token limit
            break
    
    # Use string formatting to complete the prompt
    prompt = prompt_template.format(
        "\n\n###\n\n".join(context_list),
        USER_QUESTION
    )
    return prompt

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [31]:
USER_QUESTION = """When will 5thphase of the Lok Sabha elections 2024 will take place?"""

In [32]:
prompt = basic_promt(USER_QUESTION, openai)
answer = get_response(prompt, openai)
print(answer)

I don't know


In [33]:
prompt = custom_promt(USER_QUESTION, openai)
answer = get_response(prompt, openai)
answer

'The 5thphase of the Lok Sabha elections 2024 will take place on 20 May.'

### Question 2

In [49]:
USER_QUESTION = """When will be the nominations of 1stphase of the Lok Sabha elections in 2024?"""

In [50]:
prompt = basic_promt(USER_QUESTION, openai)
answer = get_response(prompt, openai)
print(answer)

I don't know


In [51]:
prompt = custom_promt(USER_QUESTION, openai)
answer = get_response(prompt, openai)
print(answer)

The nominations of 1stphase of the Lok Sabha elections in 2024 will be conducted on 27 March.


# Conclusion

* The model responded correct information when information from a customized database was provided. 
* The LLMs are not always trained with the latest information on internet. To make an effective and incorrect queries, the existing models can be used after embedding the custome data.